# training the multiclass classification task using both VGG16 and Resnet50 

* The python file model.py contains the code for the custom written VGG16 network
* this notebook also performs training on a pretrained ResNet50 netwoek which is much more sophisticated network compared to VGG16.  * results of both models will be evalusted in the evaluation step
* in addition to the model, a train function is also written (in model.py) and imported here which is basically wrapper that performs the forward and backward loop of the train and forward loop of the eval model simultaneously.
* the function "get_loader" is a custom wrapper that makes a pytorch dataset and applies a dataloader to it.

In [6]:
import os
os.chdir('/Users/simpleai/Desktop/az_task/scripts/')
import pandas as pd
from torchvision import transforms
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
import warnings
warnings.filterwarnings('ignore')

from dataset import get_loader # custom written for this task
from model import MyVGG16      # custom written for this task
from train import train        # custom written for this task

In [15]:
# hyperparameters
NUM_CLASSES = 7
NUM_EPOCHS = 40
BATCH_SIZE = 4
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1E-4
CLASS_WEIGHT = torch.tensor([0.679, 0.764, 0.954, 0.84, 0.181, 0.971, 0.872])
device = "mps" if torch.backends.mps.is_available() else "cpu" # I'm using an M1 chip which can perform vectorized calculations over Mac GPU using ARM technology

In [10]:
# defining the transforms to be applied to the images
# transform will include on-the-fly augmentations later
transform = transforms.Compose([
   transforms.ToTensor(), 
   transforms.Resize((256,256)), # resizing to 256 by 256 due to having limited amount of ram available on my Mac
   transforms.Normalize(
       mean=[0.4914, 0.4822, 0.4465], 
       std=[0.2023, 0.1994, 0.2010]
       )
   ])

# opening the label_df where both labels and path to images are located in
label_df_path = '/Users/simpleai/Desktop/az_task/results/task1/multiclass_classification_metadata.csv'
label_df = pd.read_csv(label_df_path, index_col=0)

# dividing the label_df into train, val, test dataset
train_df = label_df.loc[label_df['type']=='Train', :] #1519 Disease; 401 Healthy 0.8:0.2 imbalance
#train_df = train_df.sample(frac=0.25) # selecting a random quarter of the data for the speed
train_df = train_df.reset_index(drop=True)

val_df = label_df.loc[label_df['type']=='Val', :]
#val_df = val_df.sample(frac=0.25) # selecting a random quarter of the data for the speed
val_df = val_df.reset_index(drop=True)

test_df = label_df.loc[label_df['type']=='Test', :]
test_df = test_df.reset_index(drop=True)

# defining the dataloader dictionary
dataloader_dict = {
    'train': get_loader(
        data_df = train_df, 
        label_cols = ['Healthy', 'DR', 'ARMD', 'MH', 'DN', 'TSLN', 'ODC'],
        batch_size = BATCH_SIZE, 
        transform = transform, 
        shuffle = True),
    'val': get_loader(
        data_df = val_df, 
        label_cols = ['Healthy', 'DR', 'ARMD', 'MH', 'DN', 'TSLN', 'ODC'],
        batch_size = BATCH_SIZE, 
        transform = transform, 
        shuffle = False),
    }

### Traing VGG16 model

In [11]:
# defining the model
model = MyVGG16(num_classes=NUM_CLASSES, input_height=256, input_width=256).to(device)


# defining criterion, optimizer
criterion = nn.CrossEntropyLoss(weight=CLASS_WEIGHT).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# training the model
model, train_val_df = train(
    model, 
    dataloader_dict, 
    criterion, 
    optimizer, 
    device, 
    checkpoint_path='/Users/simpleai/Desktop/az_task/results/task2/vgg16_multiclass_classifier.pth', 
    train_val_info_path='/Users/simpleai/Desktop/az_task/results/task2/vgg16_multiclass_classifier_info.csv',
    num_epochs=NUM_EPOCHS,
    vgg=True
    )


Epoch 1/20
__________


100%|█████████████████████████████████████████| 315/315 [04:05<00:00,  1.28it/s]


epoch 1 train >>> loss: 0.471, accuracy: 0.321


100%|█████████████████████████████████████████| 101/101 [00:38<00:00,  2.60it/s]


epoch 1 val >>> loss: 0.463, accuracy: 0.351
saving the best model in /Users/simpleai/Desktop/az_task/results/task2/vgg16_multiclass_classifier.pth
Epoch 2/20
__________


...

epoch 19 val >>> loss: 0.348, accuracy: 0.542
Epoch 20/20
__________


100%|█████████████████████████████████████████| 315/315 [03:49<00:00,  1.38it/s]


epoch 20 train >>> loss: 0.245, accuracy: 0.626


100%|█████████████████████████████████████████| 101/101 [00:38<00:00,  2.59it/s]


epoch 20 val >>> loss: 0.312, accuracy: 0.587


### Training pretrained ResNet50 model

In [16]:
# Using pretrained weights:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# modifying output number of classes from 1000 to NUM_CLASSES
model.fc = nn.Linear(2048, NUM_CLASSES)
model = model.to(device)

# defining criterion, optimizer
criterion = nn.CrossEntropyLoss(weight=CLASS_WEIGHT).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# training the model
model, train_val_df = train(
    model, 
    dataloader_dict, 
    criterion, 
    optimizer, 
    device, 
    checkpoint_path='/Users/simpleai/Desktop/az_task/results/task2/resnet50_multiclass_classifier.pth', 
    train_val_info_path='/Users/simpleai/Desktop/az_task/results/task2/resnet50_multiclass_classifier_info.csv',
    num_epochs=NUM_EPOCHS
    )

Epoch 1/40
__________


100%|█████████████████████████████████████████| 315/315 [02:31<00:00,  2.08it/s]


epoch 1 train >>> loss: 0.429, accuracy: 0.366


100%|█████████████████████████████████████████| 101/101 [00:35<00:00,  2.82it/s]


epoch 1 val >>> loss: 0.421, accuracy: 0.416
saving the best model in /Users/simpleai/Desktop/az_task/results/task2/resnet50_multiclass_classifier.pth
Epoch 2/40
__________


...

epoch 39 val >>> loss: 0.368, accuracy: 0.601
Epoch 40/40
__________


100%|█████████████████████████████████████████| 315/315 [02:51<00:00,  1.84it/s]


epoch 40 train >>> loss: 0.109, accuracy: 0.808


100%|█████████████████████████████████████████| 101/101 [00:41<00:00,  2.46it/s]

epoch 40 val >>> loss: 0.383, accuracy: 0.629
